<div align="center">

# **Limpieza**

</div>

## Librerias

In [163]:
library(dplyr)
library(tidyr)
source("../src/data/dividir_polizas_por_anio.R")
source("../src/data/ajustar_inflacion.R")
source("../src/data/Freedman_Diaconis.R")

## Data

In [164]:
df_input <- read.csv("../data/processed/datos_agrupados.csv")
df_input$X <- NULL
df_input$MARCA <- NULL
glimpse(df_input)

Rows: 5,000
Columns: 16
$ Amparo          <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, "PERDI…
$ SumaDePagos     <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, 263295…
$ Modelo          <chr> "2007_2013", "2007_2013", "2007_2013", "2007_2013", "2…
$ Color           <chr> "Otros", "PLATA", "PLATA", "BLANCO", "Otros", "PLATA",…
$ Carroceria      <chr> "SEDAN", "HATCHBACK", "SEDAN", "SEDAN", "SEDAN", "HATC…
$ Referencia1     <chr> "Otros", "ATOS [2]", "Otros", "ATOS [2]", "ACCENT VISI…
$ Referencia2     <chr> "Otros", "PRIME", "Otros", "PRIME", "GLS", "Otros", "G…
$ CLASE_FASECOLDA <chr> "AUTOMOVIL", "AUTOMOVIL", "AUTOMOVIL", "AUTOMOVIL TAXI…
$ TIPO_VEHICULO   <chr> "Livianos", "Livianos", "Livianos", "Livianos", "Livia…
$ SERVICIO        <chr> "Particular", "Particular", "Particular", "Particular"…
$ Sexo_Aseg       <chr> "F", "M", "F", "M", "F", "F", "F", NA, "F", "F", "M", …
$ Edad            <chr> "19_41", "63_85", "19_41", "Otros", "41_63", "19_41", …
$ Desde         

In [165]:
colSums(is.na(df_input))

Amparo     SumaDePagos          Modelo           Color      Carroceria 
           4320            4320               0               0               0 
    Referencia1     Referencia2 CLASE_FASECOLDA   TIPO_VEHICULO        SERVICIO 
              0               0               0               0               0 
      Sexo_Aseg            Edad           Desde           Hasta    Vr_Comercial 
            625               0               0               0               0 
           Pago 
              0

## Preparacion de los datos

Nombres de variables

In [166]:
df_input$Amparo <- gsub("DA�OS", "DAÑOS", df_input$Amparo)

Nulos en Amp

In [167]:
df_input <- df_input %>%
  mutate(
    # Convertir NA a "No aplica"
    Amparo = ifelse(is.na(Amparo), "No aplica", Amparo)
    )

Nulos en SumaDePagos

In [168]:
head(sort(unique(df_input$SumaDePagos)))

[1]     0 26248 60000 69257 70000 80000

In [169]:
  pago_minimo <- 100000
  df_input <- df_input %>% mutate(
      SumaDePagos = case_when(
        is.na(SumaDePagos) ~ 0,
        SumaDePagos < pago_minimo ~ 0,
        TRUE ~ SumaDePagos),
      # Agregar esta línea para recrear Accidentado
      Accidentado = ifelse(SumaDePagos > 0, 1, 0)
  )

El pago minimo se escogio como el valor de un espejo en 2015

Nulos en genero

In [170]:
df_input <- df_input %>% mutate(
    Sexo_Aseg = ifelse(is.na(Sexo_Aseg), "No Binarie", Sexo_Aseg)
)

In [171]:
nrow(df_input[(df_input$Sexo_Aseg == "No Binarie"), ])/nrow(df_input)

[1] 0.125

Segmentando por año

In [172]:
df_input <- dividir_polizas_por_anio(df_input)

Calculo de exposicion

In [173]:
df_input$exposicion <- as.numeric(df_input$Hasta - df_input$Desde) + 1

Ajustando a inflacion de 2015

In [174]:
max(df_input$Hasta)

[1] "2015-02-21"

In [175]:
df_input <- ajustar_inflacion_2015(df_input)

Llevamos todo a los mismo valores presentes ajustando la inflacion.

Ajustando valor comercial

In [176]:
vr_minimo <- 4000000
nrow(df_input[(df_input$Vr_Comercial > vr_minimo), ])*100/nrow(df_input)
df_input <- df_input[(df_input$Vr_Comercial > vr_minimo), ]

[1] 93.52738

El valor comercial minimo se escogio como el valor de un carro hyundai Excel en 2015

Resultados

In [177]:
colSums(is.na(df_input))

Amparo     SumaDePagos          Modelo           Color      Carroceria 
              0               0               0               0               0 
    Referencia1     Referencia2 CLASE_FASECOLDA   TIPO_VEHICULO        SERVICIO 
              0               0               0               0               0 
      Sexo_Aseg            Edad           Desde           Hasta    Vr_Comercial 
              0               0               0               0               0 
           Pago     Accidentado      exposicion 
              0               0               0

In [178]:
summary(df_input)

    Amparo           SumaDePagos          Modelo             Color          
 Length:9190        Min.   :       0   Length:9190        Length:9190       
 Class :character   1st Qu.:       0   Class :character   Class :character  
 Mode  :character   Median :       0   Mode  :character   Mode  :character  
                    Mean   :  272939                                        
                    3rd Qu.:       0                                        
                    Max.   :35206209                                        
  Carroceria        Referencia1        Referencia2        CLASE_FASECOLDA   
 Length:9190        Length:9190        Length:9190        Length:9190       
 Class :character   Class :character   Class :character   Class :character  
 Mode  :character   Mode  :character   Mode  :character   Mode  :character  
                                                                            
                                                                            

## Exportando los resultados

In [179]:
write.csv(df_input, "../data/processed/datos_limpios.csv", row.names = FALSE)